<a href="https://colab.research.google.com/github/kolla118/KOLLA_INFO5731_Spring2020-/blob/main/INFO5731_Assignment_Two_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(40 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect the top 10000 User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the reviews of the top 100 most popular software from [G2](https://www.g2.com/) or [Capterra](https://www.capterra.com/)

(4) Collect the abstracts of the top 10000 research papers by using the query [natural language processing](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(5) Collect all the information of the 904 narrators in the [Densho Digital Repository](https://ddr.densho.org/narrators/).

(6) Collect the top 10000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter. 


In [6]:
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import pandas as pd

HEADERS = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
            AppleWebKit/537.36 (KHTML, like Gecko) \
            Chrome/90.0.4430.212 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})


def get_customer_reviews(soup):
  str = ""

  scraped_review_list = soup.find_all("div",{"data-hook":"review"})
  #print(scraped_review_list)
  
  for review in scraped_review_list:
    str = str + review.get_text()
  #print(str)
  return str.split("\n")


def get_documents(url):
  
  raw_html =  requests.get(url, headers=HEADERS).text
  #print(raw_html)
  soup = BeautifulSoup(raw_html, 'lxml')
  return get_customer_reviews(soup)

iphone_review_list = []

for i in range(0,288):
  documents = get_documents("https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/product-reviews/B07ZPKF8RG/ref=cm_cr_getr_d_paging_btm_next_{page_no}?ie=UTF8&reviewerType=all_reviews&pageNumber={page_no}".format(page_no = i))
  iphone_review_list.extend(documents)
  #print(len(documents))
  #print(documents)
print(iphone_review_list)
df = pd.DataFrame()
df["Reviews"] = iphone_review_list
df.to_csv('iphone_reviews.csv', index=False)




[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['Katherine1.0 out of 5 stars', 'Not FULLY Unlocked', 'Reviewed in the United States on February 13, 2020Size: 64GBColor: BlackService Provider: UnlockedProduct grade: RenewedVerified Purchase', '', "  Purchased this product advertised as fully unlocked. Inserted my sim card and it did not work. Contacted my service provider and they checked my IMEI number and it reported that the phone was still locked by the original carrier. Phone came in good condition just couldn't use any service provider.", '', '', '432 people found this helpful', '', '          Helpful', '', '', 'Report abuse', '', 'Showing 0 comments', 'There was a problem loading comments right now. Please try again later.brenda4.0 out of 5 stars', 'NOT EXPECTED, GREAT PURCHASE!', 'Reviewed in th

# **Question 2**

(30 points). Write a python program to **clean the text data** you collected above and save the data in a new column in the csv file. The data cleaning steps include:

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the [stopwords list](https://gist.github.com/sebleier/554280).

(4) Lowercase all texts

(5) Stemming. 

(6) Lemmatization.

In [7]:
import os
import tweepy as tw
import pandas as pd
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer

words = set(nltk.corpus.words.words())

def get_lemmatized_docs(X):
  stemmer = WordNetLemmatizer()
  eng_stop_word_list = stopwords.words("english")
  bag_of_words = []
  for sen in range(0, len(X)):
      # removing all the special characters
      document = re.sub(r'\W', ' ', str(X[sen]))

      # removing all single characters
      document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

      # removing multiple spaces and adding single space in place
      document = re.sub(r'\s+', ' ', document, flags=re.I)

      # removing the 'b' prefix added during byte conversion
      document = re.sub(r'^b\s+', '', document)

      # removing numbers
      document = re.sub(r'\d+','',document)

      # converting all words to lowercase
      document = document.lower()
      
      # Splitting the cleaned document into a list of words, which will be subsequently added to a bag after more processing
      document = document.split()
      document = [stemmer.lemmatize(word) for word in document]
      document = [word for word in document if word not in eng_stop_word_list]
      document = ' '.join(document)

      document = " ".join(word for word in nltk.wordpunct_tokenize(document) if word.lower() in words or not word.isalpha())

      #Bag of words
      bag_of_words.append(document)

  return bag_of_words

df = pd.read_csv('iphone_reviews.csv')

clean_docs  = get_lemmatized_docs(df["Reviews"])
print(len(clean_docs))
print(clean_docs)







[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
58992
['star', 'fully unlocked', 'united state size provider grade purchase', 'nan', 'product fully unlocked inserted card work service provider checked number phone wa still locked original carrier phone came good condition use service provider', 'nan', 'nan', 'people found helpful', 'nan', 'helpful', 'nan', 'nan', 'report abuse', 'nan', 'showing comment', 'wa problem loading comment right please try later star', 'great purchase', 'united state august size provider grade purchase', 'nan', 'wa feeling bit skeptical order wa proven wrong phone look brand new besides minimal light scratch almost impossible notice battery life still also like wa screen prote

# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [8]:
import nltk
import pandas as pd
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize, pos_tag, pos_tag_sents
df = pd.read_csv('iphone_reviews.csv')
clean_df = df[df["Reviews"].notnull()]

clean_list =  pos_tag_sents( clean_df["Reviews"].map(lambda x : str(x)).apply(word_tokenize).tolist() )
#print(clean_list)
tag_list = clean_list

nouns = 0
for tag in tag_list:
  for x,y in tag:
    if(y.startswith('N')):
       nouns=nouns+1
noun_count = "nouns:{}".format(nouns)
print(noun_count)


verbs = 0
for tag in tag_list:
  for x,y in tag:
    if(y.startswith('V')):
       verbs=verbs+1
verbs_count = "verbs:{}".format(verbs)
print(verbs_count)


adj = 0
for tag in tag_list:
  for x,y in tag:
    if(y.startswith('J')):
       adj=adj+1
adj_count = "adjectives:{}".format(adj)
print(adj_count)


import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp_mod = spacy.load("en_core_web_sm")

df['Reviews']
list = df['Reviews'].tolist()
list[0]

for x in list:
   if not x:
     doc = (nlp_mod(x))
     print([(X.text, X.label_) for X in doc.ents])








[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
nouns:92952
verbs:34320
adjectives:21076


## **Write your explanations of the constituency parsing tree and dependency parsing tree here (Question 3-2):** 

Constituency parsing prases a sentence based on sub-phrases, which can belong to any category of grammar like verbs, nouns etc.,

Dependency parsing analyzes the grammatical structure of a sentence by examining the relationship between the words of a sentence.



In [ ]:
Part 3

In [10]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp_mod = spacy.load("en_core_web_sm")

df['Reviews']
list = df['Reviews'].tolist()
list[0]

for x in list:
   if not x:
     doc = (nlp_mod(x))
     print([(X.text, X.label_) for X in doc.ents])